# TorchSparse for MMDetection3D Plugin Demo
You can run the cells below to run the evaluation of TorchSparse integrated MMDetection3D models. 

## Dependencies
- MMDetection3D installation: Please follow the [MMDetection3D documentation](https://mmdetection3d.readthedocs.io/en/latest/get_started.html). 
- Pre-process the datasets required by MMDetection3D ([see here](https://mmdetection3d.readthedocs.io/en/latest/user_guides/dataset_prepare.html)). 
- TorchSparse installation. 
- Install TorchSparse plugin for MMDetection3D
    1. Clone this repository
    2. Go to `examples/mmdetection3d` and run `pip install -v -e .`

## Notes
1. For model evaluation, you need to change the data root in the original mmdetection3d's model config to be the full path of the corresponding dataset root. The default is the relative path because mmdet3d expect you to run the evaluation under their repository folder. However, to run this demo, the relative path won't work and you need to change it to the full path. 

# Steps
1. Install the dependencies. 
2. Specify the base pathes and model registry. 
3. Activate the plugin: In `mmdetection3d/tools/test.py`, add `import ts_plugin` as the last import statement.  
4. Run demo. 
5. Print the evaluation results. 

# Lists of Supported Models
- SECOND
- PV-RCNN
- CenterPoint
- PartA2

# The Actual Part
## Load the Weight Conversion Module
The dimensions of TorchSparse differs from the SpConv, so the parameter dimension conversion is required to use the TorchSparse backend. The following cell loads the converter. 

In [ ]:
import importlib.util
import sys, os
from pathlib import Path
import subprocess

# Define the relative path to the file
relative_path = "../converter.py"
file_path = Path().resolve() / relative_path

# Add the directory containing the file to sys.path
sys.path.append(str(file_path.parent))

# Load the module
spec = importlib.util.spec_from_file_location("convert_weights", str(file_path))
converter = importlib.util.module_from_spec(spec)
spec.loader.exec_module(converter)

converter = getattr(converter, "convert_weights")

In [ ]:
# Dummy check for whether the weight converter is successfully loaded. 
print(converter)

## Specify the Paths and Environment Parameters
To run this demo, you need to provide the following paths:
1. `mmdet3d_path`: MMDetection3D installation path. We need this path to find the `test.py` evaluation script. 
2. `mmdet3d_model_base_path`: Input pretrained weight path. the pretrained weights you download from the MMDetection3D model zoo should be put under a same base folder. 
3. `torchsparse_model_base_path`: Output pretrained weight path. The converted weights for various models should be put under the same base folder as well. 
4. `mmdet3d_cfg_base_path`: MMDetection3D configuration files base path. This configuration file is required in the model conversion. Specifically, it use the original configuration file to create a model to identify the Sparse Conv modules, and convert the weights for only those modules. By default, if you installed the mmdet3d in development mode with `-e` then this should just be the `config` folder in the mmdet3d repo. 
5. Conda environment name: this demo initialize a sub-shell to execute the demo with `subprocess`. So you need to specify the name of the conda environment that you want to use to run the demo. 

For paths 2, 3, and 4, we expect you to organize them by having a base path and put the checkpoint/configurations files of different models under the same basepath. For example, for the input pertrained weight path, the file structure looks like: 

```text
mmdet_model_base_folder/                      
├── SECOND/                 
│   └── SECOND_Checkpoint.pth
├── PV-RCNN/
│   └── PV-RCNN_Checkpoint.pth
└── CenterPoint/
    └── CenterPoint_Checkpoint.pth
```
To configure the path for SECOND demo, you need to configure the `mmdet3d_model_base_path` to the path of the folder `mmdet_model_base_folder` and in the SECOND's registry entry, set `ckpt_before` to be `SECOND/SECOND_Checkpoint.pth`. 

In addition to the paths, we also need you to specify:
1. SpConv version of the original model.
2. `cfg_options`: some modules in the model is replaced by the TorchSparse layers. When running the evaluation, you don't need to provide a new configuration file to specify the use of TorchSparse layers. You can rather use the original mmdetection3d config file but use the `cfg_options` to replace certain modules to use the TorchSparse module. Typically, only one or two modules needed to be replaced. You can see the specific usage from the exaple below. 
3. Name of the conda environment the dependencies is installed. 


In [4]:
env_name = "torchsparse"

# Please complete the following base paths. 
base_paths = {
    'mmdet3d_path': None,
    'mmdet3d_model_base_path': None,
    'torchsparse_model_base_path': os.path.join(os.path.abspath(''), "converted_models"),
    'mmdet3d_cfg_base_path': None
}

# Specify the model specific path and registry values. 
# NOTE: ckpt_before is associated with the mmdet3d_model_base_path and ckpt_after is associated with the torchsparse_model_base_path. 
second_3d_car = {
    'ckpt_before': 'SECOND/second_hv_secfpn_8xb6-80e_kitti-3d-car-75d9305e.pth',
    'ckpt_after': 'SECOND/second_hv_secfpn_8xb6-80e_kitti-3d-car-75d9305e.pth',
    'cfg_path': 'second/second_hv_secfpn_8xb6-80e_kitti-3d-car.py',
    'v_spconv': 2,
    'cfg_options': "--cfg-options test_evaluator.pklfile_prefix=outputs/torchsparse/second --cfg-options model.middle_encoder.type=SparseEncoderTS"
}


In [7]:
base_paths

{'mmdet3d_path': '/home/yingqi/repo/mmdetection3d',
 'mmdet3d_model_base_path': '/home/yingqi/repo/mmdetection3d/models',
 'torchsparse_model_base_path': '/home/yingqi/repo/torchsparse-dev/examples/mmdetection3d/converted_models',
 'mmdet3d_cfg_base_path': '/home/yingqi/repo/mmdetection3d/configs'}

The function to run a single demo is defined below. Based on the configuration dictionary you provid, it convert the model weights then use the `tools/test.py` in the `mmdetection3d` repo to run the model evaluation. 

In [10]:
def mmdet3d_single_demo(registry_entry, base_paths, convert=True):
    """Run Single Model Demo

    :param registry_entry: the model demo registry. 
    :type registry_entry: dict
    :param base_paths: the base paths. 
    :type base_paths: dict
    :param convert: whether to convert the model. If set to false, it skip the model conversion and use the provided checkpoint to run model evaluation directly. Defaults to True.
    :type convert: bool, optional
    :return: return the process object that used to run the demo. 
    :rtype: CompletedProcess
    """

    assert os.path.isdir(base_paths['mmdet3d_path']), "Please specify the mmdet3d_path in the base_paths."
    assert os.path.isdir(base_paths['mmdet3d_model_base_path']), "Please specify the mmdet3d_model_base_path in the base_paths."
    assert os.path.isdir(base_paths['torchsparse_model_base_path']), "Please specify the torchsparse_model_base_path in the base_paths."
    assert os.path.isdir(base_paths['mmdet3d_cfg_base_path']), "Please specify the mmdet3d_cfg_base_path in the base_paths."

    # pre-process paths
    cfg_path = os.path.join(base_paths['mmdet3d_cfg_base_path'], registry_entry['cfg_path'])
    test_file_path = os.path.join(base_paths['mmdet3d_path'], "tools/test.py")
    mmdet3d_model_path = os.path.join(base_paths['mmdet3d_model_base_path'], registry_entry['ckpt_before'])
    assert os.path.isdir(base_paths['torchsparse_model_base_path']), "Please create the directory for the converted model."
    torchsparse_model_path = os.path.join(base_paths['torchsparse_model_base_path'], registry_entry['ckpt_after'])
    
    cfg_options = registry_entry['cfg_options']
    # convert the model
    if convert:
        parent_dir = os.path.dirname(torchsparse_model_path)
        if not os.path.exists(parent_dir):
            os.makedirs(parent_dir)
        converter(
            ckpt_before=mmdet3d_model_path,
            ckpt_after=torchsparse_model_path,
            cfg_path=cfg_path,
            v_spconv = registry_entry['v_spconv']
        )

    command = f'bash -c "conda activate {env_name}; python {test_file_path} {cfg_path} {torchsparse_model_path} {cfg_options} --task lidar_det"'
    print(command)
    result = subprocess.run(command, capture_output=True, text=True, shell=True, executable='/bin/bash')
    return result  # result have .stdout and .stderr attributes to get the output. 
    

## Evaluate MMDetection3d Models

### SECOND
Run a SECOND demo. You can print the evaluation results of the model from the sub-process's `stdout` and `stderr`. 


In [ ]:
second_results = mmdet3d_single_demo(second_3d_car, base_paths, convert=True)
print(second_results.stderr)
print(second_results.stdout)

Expected Output: 

```
----------- AP11 Results ------------

Car AP11@0.70, 0.70, 0.70:
bbox AP11:95.2015, 89.6519, 88.0073
bev  AP11:89.9621, 87.2725, 84.2825
3d   AP11:88.3629, 78.2199, 76.0327
aos  AP11:94.94, 89.08, 87.23
Car AP11@0.70, 0.50, 0.50:
bbox AP11:95.2015, 89.6519, 88.0073
bev  AP11:95.3329, 89.9520, 88.7400
3d   AP11:95.2805, 89.8595, 88.5336
aos  AP11:94.94, 89.08, 87.23

----------- AP40 Results ------------

Car AP40@0.70, 0.70, 0.70:
bbox AP40:97.4063, 92.4550, 89.2481
bev  AP40:92.6387, 88.4049, 85.2355
3d   AP40:90.4511, 81.3433, 76.1927
aos  AP40:97.13, 91.81, 88.42
Car AP40@0.70, 0.50, 0.50:
bbox AP40:97.4063, 92.4550, 89.2481
bev  AP40:97.5160, 94.7415, 91.7295
3d   AP40:97.3701, 94.5687, 91.4920
aos  AP40:97.13, 91.81, 88.42
```

### PV-RCNN
Run a PV-RCNN Demo.

In [9]:
# PV-RCNN Registry
pv_rcnn_config = {
    "ckpt_before": "PV-RCNN/pv_rcnn_8xb2-80e_kitti-3d-3class_20221117_234428-b384d22f.pth",
    "ckpt_after": "PV-RCNN/pv_rcnn_8xb2-80e_kitti-3d-3class_20221117_234428-b384d22f.pth",
    "cfg_path": "pv_rcnn/pv_rcnn_8xb2-80e_kitti-3d-3class.py",
    "v_spconv": 1,
    "cfg_options": "--cfg-options test_evaluator.pklfile_prefix=outputs/torchsparse/pv_rcnn --cfg-options model.middle_encoder.type=SparseEncoderTS --cfg-options model.points_encoder.type=VoxelSetAbstractionTS"
}

In [ ]:
pv_rcnn_results = mmdet3d_single_demo(pv_rcnn_config, base_paths, convert=True)
print(pv_rcnn_results.stderr)

In [ ]:
print(pv_rcnn_results.stdout)

Expected Output: 

```
----------- AP11 Results ------------

Pedestrian AP11@0.50, 0.50, 0.50:
bbox AP11:74.1319, 68.4703, 65.9149
bev  AP11:68.2026, 62.7491, 57.1043
3d   AP11:66.6080, 59.7569, 55.1617
aos  AP11:68.98, 63.64, 60.68
Pedestrian AP11@0.50, 0.25, 0.25:
bbox AP11:74.1319, 68.4703, 65.9149
bev  AP11:80.5667, 76.2589, 72.7974
3d   AP11:80.3568, 76.0134, 72.2977
aos  AP11:68.98, 63.64, 60.68
Cyclist AP11@0.50, 0.50, 0.50:
bbox AP11:89.2310, 82.0387, 77.1643
bev  AP11:87.8058, 74.9448, 70.5274
3d   AP11:87.2027, 73.2608, 69.6121
aos  AP11:89.13, 81.69, 76.77
Cyclist AP11@0.50, 0.25, 0.25:
bbox AP11:89.2310, 82.0387, 77.1643
bev  AP11:88.6302, 80.1792, 74.8060
3d   AP11:88.6302, 80.1792, 74.8060
aos  AP11:89.13, 81.69, 76.77
Car AP11@0.70, 0.70, 0.70:
bbox AP11:96.0265, 89.5369, 89.1852
bev  AP11:90.1265, 88.0958, 87.6436
3d   AP11:89.2321, 83.7058, 78.7935
aos  AP11:95.98, 89.43, 89.03
Car AP11@0.70, 0.50, 0.50:
bbox AP11:96.0265, 89.5369, 89.1852
bev  AP11:96.1496, 94.8182, 89.2712
3d   AP11:96.0921, 89.5371, 89.2317
aos  AP11:95.98, 89.43, 89.03

Overall AP11@easy, moderate, hard:
bbox AP11:86.4631, 80.0153, 77.4215
bev  AP11:82.0450, 75.2632, 71.7584
3d   AP11:81.0143, 72.2412, 67.8558
aos  AP11:84.70, 78.25, 75.49

----------- AP40 Results ------------

Pedestrian AP40@0.50, 0.50, 0.50:
bbox AP40:75.6494, 69.7741, 66.0890
bev  AP40:69.5448, 62.1173, 57.1881
3d   AP40:66.6659, 59.2055, 54.1700
aos  AP40:70.00, 64.19, 60.31
Pedestrian AP40@0.50, 0.25, 0.25:
bbox AP40:75.6494, 69.7741, 66.0890
bev  AP40:82.8723, 78.0379, 73.2982
3d   AP40:82.6538, 77.1948, 72.8713
aos  AP40:70.00, 64.19, 60.31
Cyclist AP40@0.50, 0.50, 0.50:
bbox AP40:93.8638, 84.2218, 80.1001
bev  AP40:92.8451, 75.6214, 71.7649
3d   AP40:90.3880, 73.2361, 69.4116
aos  AP40:93.71, 83.84, 79.61
Cyclist AP40@0.50, 0.25, 0.25:
bbox AP40:93.8638, 84.2218, 80.1001
bev  AP40:93.9661, 81.6019, 77.2742
3d   AP40:93.9661, 81.6019, 77.2742
aos  AP40:93.71, 83.84, 79.61
Car AP40@0.70, 0.70, 0.70:
bbox AP40:97.8348, 94.5482, 94.0081
bev  AP40:94.4796, 90.7830, 88.6291
3d   AP40:91.8635, 84.5625, 82.4022
aos  AP40:97.80, 94.41, 93.80
Car AP40@0.70, 0.50, 0.50:
bbox AP40:97.8348, 94.5482, 94.0081
bev  AP40:97.9316, 96.4609, 94.4074
3d   AP40:97.8820, 94.6416, 94.3069
aos  AP40:97.80, 94.41, 93.80

Overall AP40@easy, moderate, hard:
bbox AP40:89.1160, 82.8480, 80.0657
bev  AP40:85.6232, 76.1739, 72.5274
3d   AP40:82.9724, 72.3347, 68.6612
aos  AP40:87.17, 80.81, 77.91

```

### CenterPoint Voxel 0.1 Circular NMS

This is the first NuScenes model in the demo. Please remember to update the path of the NuScenes dataset in the mmdetection3d dataset config `configs/_base_/datasets/nus-3d.py`.

In [ ]:
centerpoint_config = {
    "ckpt_before": "CenterPoint/centerpoint_01voxel_second_secfpn_circlenms_4x8_cyclic_20e_nus_20220810_030004-9061688e.pth",
    "ckpt_after": "CenterPoint/centerpoint_01voxel_second_secfpn_circlenms_4x8_cyclic_20e_nus_20220810_030004-9061688e.pth",
    "cfg_path": "centerpoint/centerpoint_voxel01_second_secfpn_head-circlenms_8xb4-cyclic-20e_nus-3d.py",
    "v_spconv": 1,
    "cfg_options": "--cfg-options model.pts_middle_encoder.type=SparseEncoderTS"
}

centerpoint_results = mmdet3d_single_demo(centerpoint_config, base_paths, convert=True)
print(centerpoint_results.stderr)


In [ ]:
print(centerpoint_results.stdout)

In [ ]:
print(centerpoint_results.stdout)

Expected Outputs: 

```
Evaluating bboxes of pred_instances_3d
mAP: 0.5544
mATE: 0.2988
mASE: 0.2538
mAOE: 0.3110
mAVE: 0.3039
mAAE: 0.1977
NDS: 0.6407
Eval time: 53.4s

Per-class results:
Object Class	AP	ATE	ASE	AOE	AVE	AAE
car	0.845	0.186	0.152	0.113	0.304	0.194
truck	0.522	0.324	0.185	0.126	0.283	0.240
bus	0.667	0.354	0.181	0.062	0.535	0.268
trailer	0.362	0.546	0.207	0.447	0.208	0.164
construction_vehicle	0.160	0.639	0.414	0.858	0.117	0.334
pedestrian	0.827	0.165	0.276	0.410	0.244	0.101
motorcycle	0.529	0.213	0.237	0.292	0.511	0.264
bicycle	0.341	0.169	0.268	0.421	0.229	0.016
traffic_cone	0.638	0.162	0.342	nan	nan	nan
barrier	0.653	0.230	0.277	0.070	nan	nan
```